Comece importando as bibliotecas necessárias:
- pyspark.sql
- random
- faker
- pymysql para lidar com a conexão ao banco de dados MariaDB.

Inicialize uma sessão Spark, que é a maneira de interagir com os dados no Spark

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[1]") \
                    .appName('test') \
                    .config("spark.jars", "mariadb-java-client-1.8.0.jar")\
                    .getOrCreate()
spark

23/12/22 23:37:29 WARN Utils: Your hostname, workstation-330S-14IKB resolves to a loopback address: 127.0.1.1; using 10.0.0.31 instead (on interface wlp1s0)
23/12/22 23:37:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/12/22 23:37:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/22 23:37:30 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/12/22 23:37:30 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
mysql_url = "jdbc:mysql://localhost:3306/teste"
tabela =  "tera"
properties = {
    "url": mysql_url,
    "table": tabela,
    "numPartitions": 10,
    "properties":{
        "user": "root",
        "password": "secret",
        "driver": "org.mariadb.jdbc.Driver"
    }
    
}

df = spark.read.jdbc(**properties).cache()
df.show(1)

23/12/22 23:37:38 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---------------------------+-------------+--------------------+-----+-------------+-------+-----------------------+-------------------------+--------------------+--------------------+---------+--------------------+-------------+-------------+---------------------------------+-----------------------+----------------------+------------------------+------------------------------+-------------------------+------------------------+---------------+-----------------------+----------------------------+--------------+----------------------+-------------------+---------------------+--------------------------+----------------------------------+-----------------------------------+-------------------------------------+------------------------------+----------------------------+-----------------------------+---------------------------------------+----------------------+---------------------------------+--------------------------+-------------------------------------+-----------------------------+--

In [3]:
import json
with open('param.json', 'r') as file:
    data = json.load(file)

print(json.dumps(data))
parametros_carga = data['parametros_carga']

for file in data['arquivo_sql']:
    with open(file, 'r') as file:
        query = file.read() 
    query = query.format_map(parametros_carga)  
    df.createOrReplaceTempView(tabela)
    df_stg = spark.sql(query)

df_stg.show()

{"arquivo_sql": ["count.sql"], "parametros_carga": {"tabela": "tera", "cidade": "Geneve"}}
+------+
|  City|
+------+
|Geneve|
+------+

